In [54]:
#Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
from citipy import citipy
import random
import csv
import requests as req
import time
import numpy as np
import json

In [55]:
api_key = "7bb4964424c64fe8ddd15329382d570d"

In [56]:
# Latitude and longitude zones based on geographic coordinate system 
lat_zone = np.arange(-90, 90, 10)
lon_zone = np.arange(-180, 180, 10)

# Create dataframe for storing data
cities_df = pd.DataFrame()

# Make columns in dataframe
cities_df["Latitude"] = ""
cities_df["Longitude"] = ""
cities_df["City"] = ""
cities_df["Country"] = ""

# Create 'for' loop to get latitude and longitude values
for coord_lat in lat_zone:
    
    for coord_lon in lon_zone:
        # Get list of latitude and longitude values (0.01 for value to 2 decimal places)
        lat = list(np.arange(coord_lat, coord_lat + 15, 0.01))
        lon = list(np.arange(coord_lon, coord_lon + 15, 0.01))
        
        # Random latitude and longitude values 
        random_lats = random.sample(lat, 50)
        random_lons = random.sample(lon, 50)
        
        # Sample of latitude and longitude values
        lat_samp = [coord_lat + lat for lat in random_lats]
        lon_samp = [coord_lon + lon for lon in random_lons]
        
        # Store values in dataframe created
        cities_df = cities_df.append(pd.DataFrame.from_dict({
            "Latitude": lat_samp, 
            "Longitude": lon_samp}))

cities_df = cities_df.reset_index(drop=True)
cities_df.head()

C:\Users\jhoff\Anaconda3.6\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


,City,Country,Latitude,Longitude
0,NaN,NaN,-179.78,-346.31
1,NaN,NaN,-178.05,-350.83
2,NaN,NaN,-178.19,-358.38
3,NaN,NaN,-168.37,-357.40
4,NaN,NaN,-169.35,-347.02


In [ ]:
# Use 'for' loop to locate the neareast city based on latitude and longitude stored in dataframe
for column, row in cities_df.iterrows():
    city = citipy.nearest_city(row["Latitude"], row["Longitude"])
    cities_df.set_value(column, "City", city.city_name)
    cities_df.set_value(column, "Country", city.country_code)

# Remove Latitude and Longitude columns to get city names and countries only
new_cities_df = cities_df.drop(["Latitude", "Longitude"], axis=1)

# Remove duplicate cities and keep unique cities and countries only
new_cities_df = new_cities_df.drop_duplicates()
new_cities_df.head()

C:\Users\jhoff\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
C:\Users\jhoff\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


In [ ]:
# Choose 500 random cities based on longitude and latitude
random_cities_df = new_cities_df.sample(500)

# Reset index
random_cities_df = random_cities_df.reset_index(drop=True)
random_cities_df.head()

In [ ]:
 url = "http://api.openweathermap.org/data/2.5/weather"

params = {"appid": "2ec88724f3fad131ccffe1dfb2efab45",
          "units": "metric"}

# Use 'for' loop to retrieve weather info by rows in dataframe
for column, row in random_cities_df.iterrows():
    
    # Reference rows for 'q' param
    params["q"] = f'{row["City"]}, {row["Country"]}'
    
    # Get and print links for each city
    print(f'Weather information for {params["q"]}')
    weather_response = req.get(url, params)
    print(weather_response.url)
    weather_response  = weather_response.json()
    
    # Get weather data and input into dataframe
    random_cities_df.set_value(column, "Latitude", weather_response.get("coord", {}).get("lat"))
    random_cities_df.set_value(column, "Longitude", weather_response.get("coord", {}).get("lon"))
    random_cities_df.set_value(column, "Temperature", weather_response.get("main", {}).get("temp_max"))
    random_cities_df.set_value(column, "Wind Speed", weather_response.get("wind", {}).get("speed"))
    random_cities_df.set_value(column, "Humidity", weather_response.get("main", {}).get("humidity"))
    random_cities_df.set_value(column, "Cloudiness", weather_response.get("clouds", {}).get("all"))

In [ ]:
# Display dataframe with all info
random_cities_df

# Remove cities with no data available
new_random_cities_df = random_cities_df.dropna()
new_random_cities_df.head()

In [ ]:
# Save dataframe to csv file
new_random_cities_df.to_csv("WeatherInfo.csv")

In [ ]:
#Temperature (F) vs. Latitude
temp_vs_lat = new_random_cities_df.plot(kind="scatter",
                                        x="Latitude",
                                        y="Temperature",
                                        color="blue",
                                        edgecolor="black",
                                        grid=True)
plt.title("Latitude of City vs Temperature 07/07/2018")
plt.xlabel("Latitude")
plt.ylabel("Temperature (F)")
plt.savefig("TempVsLat.png")
plt.show()

In [ ]:
#Humidity vs. Latitude
humidity_vs_lat = new_random_cities_df.plot(kind="scatter",
                                         x="Latitude",
                                         y="Humidity",
                                         color="blue",
                                         edgecolor="black",
                                         grid=True)
plt.title("Latitude of City vs Humidity 07/07/2018")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.savefig("HumidityVsLat.png")
plt.show()

In [ ]:
#Cloudiness vs. Latitude
cloud_vs_lat = new_random_cities_df.plot(kind="scatter",
                                         x="Latitude",
                                         y="Cloudiness",
                                         color="blue",
                                         edgecolor="black",
                                         grid=True)
plt.title("Latitude of City vs Cloudiness 07/07/2018")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.ylim(-20, 120)
plt.savefig("CloudinessVsLat.png")
plt.show()

In [ ]:
#Wind Speed (mph) vs. Latitude
windspeed_vs_lat = new_random_cities_df.plot(kind="scatter",
                                         x="Latitude",
                                         y="Wind Speed",
                                         color="blue",
                                         edgecolor="black",
                                         grid=True)
plt.title("Latitude of City vs Wind Speed 07/07/2018")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.ylim(-2, 14)
plt.savefig("WindSpeedVsLat.png")
plt.show()